sweep알고리즘
코드 및 프로그램에 대한 권한은 장두혁,최수찬 한테 있습니다.


In [ ]:
#!git clone https://github.com/justin95214/sweep.git

In [ ]:
import numpy as np
import pandas as pd

import io
from google.colab import files
myfile = files.upload()

Saving DP_raw.xlsx to DP_raw.xlsx


In [ ]:
df_DP = pd.read_excel(io.BytesIO(myfile['DP_raw.xlsx']))

In [ ]:
myfile = files.upload()

Saving PR_.xlsx to PR_.xlsx


In [ ]:
df_client = pd.read_excel(io.BytesIO(myfile['PR_.xlsx']))

In [ ]:
myfile = files.upload()

Saving sheet_4.xlsx to sheet_4.xlsx


In [ ]:
df_info  = pd.read_excel(io.BytesIO(myfile['sheet_4.xlsx']))

In [ ]:
df_info

,구분,대당 적재능력(box),대당 운행시간(분),평균 운행 속도(km/h),상차시간(분),하차시간(분),DP
0,기준 데이터,100,360,50,5,5,부발
1,기준 데이터,100,360,50,5,5,안양
2,기준 데이터,100,360,50,20,15,북부
3,기준 데이터,100,360,50,20,15,강서
4,기준 데이터,100,360,50,20,15,인천
5,기준 데이터,100,360,50,20,15,남부
6,기준 데이터,100,360,50,20,15,서부
7,기준 데이터,100,360,50,20,15,강릉
8,기준 데이터,100,360,50,20,15,춘천
9,기준 데이터,100,360,50,20,15,대구


In [ ]:
week_num =['월','화','수','목','금']
for day in week_num:
  %mkdir $day

In [ ]:
# df-client info
df_client = pd.read_excel('PR_.xlsx')
#print(df_client.head(10))

# DP address
df_DP = pd.read_excel('DP_raw.xlsx')
#print(df_DP.head(5))
df_client = pd.merge(left=df_client, right=df_DP, on="DP", how="right")
df_client = df_client
# DF truck & time info df
df_info = pd.read_excel('sheet_4.xlsx')
# merge the info
info_col=df_info.columns[1:]

#df_client[info_col] = df_info[info_col].iloc[0]
df_client = pd.merge(left=df_client, right=df_info, on="DP", how="right")

In [ ]:
week_num =['월','화','수','목','금']


# change the day
for day in week_num:
	print("############################################################################################################\n")
	#Per DP, get the client info
	for DF_element in  df_DP['DP'].values.tolist():
		print("@@@@@@@@@@@@@@@@@@@@@@@@		["+ DF_element+"/"+day+"]		@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n")
		#지역 요일 별 df
		temp_df = df_client.loc[df_client['DP']==DF_element]
		filter_columns = list(set(temp_df.columns.values.tolist())- set(week_num))
		filter_columns.append(day)
		temp_df = temp_df[filter_columns]	
		temp_df['stopby'] = 0
		temp_df =temp_df.dropna(axis=0)
		print("지역 요일 별 df shape : ",temp_df.shape)
		#print(temp_df.head(5))
		#stopby check!!
		#temp_df.to_csv("./result/test_"+DF_element+"_"+day+".csv",index=False ,encoding='utf-8-sig')
		# how many truck we need!
		# how many back to DP
		Point_list =['DP']
		Rest_time_list =[]
		Rest_Stock_list=[]
		distance = 0
		print(temp_df.iloc[0]['대당 적재능력(box)'])
		Rest_Stock_list.append(temp_df.iloc[0]['대당 적재능력(box)'])
		Rest_time_list.append(temp_df.iloc[0]['대당 운행시간(분)'])
		current_point = 'DP'
		truck_list = []
		check = 0
		up_time = 0
		d_list=[]
		time_list=[]
		one_truck_list=[]
		Max_row =0
		col_list=[]
		T_time_list = []
		DP_count = []

		while(len(temp_df.loc[temp_df['stopby']==0]) >0):
			print("Point_list : ", Point_list)
			#print("---------------------------")
			#print("Time list :", Rest_time_list)
			#print("---------------------------")
			#print("Stock list :", Rest_Stock_list)
			#print("---------------------------")
			#check = check+1
			print("---------------------------------------------------------------------------------------------------------------------------------\n")
			if current_point =='DP':

				temp_df['DP_x곡률값']=temp_df['Latitude']*3600
				temp_df['DP_y곡률값']=temp_df['Longitude']*3600
	
				#client curvature value
				temp_df['x곡률값']=temp_df['경도(X좌표)']*3600
				temp_df['y곡률값']=temp_df['위도(Y좌표)']*3600

				temp_df['DP_위도거리']= abs(temp_df['x곡률값']-temp_df['DP_y곡률값'])*0.0245
				temp_df['DP_경도거리']= abs(temp_df['y곡률값']-temp_df['DP_x곡률값'])*0.0306
				temp_df['DP_운행거리'] = temp_df['DP_위도거리']+temp_df['DP_경도거리']

				#최초와 중간 회차 뒤에 경우 
				Max_distance_row = temp_df[temp_df['stopby']==0].sort_values(by='DP_운행거리', ascending=False).iloc[0]
				#print(">>운행거리 :", Max_distance_row['DP_운행거리'])

				#현재 위치 변경
				current_point = Max_distance_row['code']
				#print(check,"거리로 선정 Client: ",Max_distance_row['DP_운행거리'],current_point)
				print(">>",current_point)

				#장소 리스트 업데이트 
				Point_list.append(current_point)

				#stopby =1 check
				temp_df.loc[(temp_df['code']==current_point),'stopby']=1

				#적재량 리스트 업데이트
				current_point_stock = Max_distance_row[day].tolist()

				Rest_Stock_list.append(Rest_Stock_list[-1]-current_point_stock)
				
				print(" >> currrent_p : ",current_point,"	current_stock : ",current_point_stock)
				dist = temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]
				fast = temp_df.loc[temp_df['code']==current_point]['평균 운행 속도(km/h)'].values.tolist()[0]
				time =  (dist / fast)*60
				print("dist :	",dist,"/ fast :	",fast,"/ time :	",time)
							

				
				#남은 시간 리스트  업데이트 
				#소요 시간 = 상차 + [DP-현재 위치]이동시간 + 하차  
				Rest_time_list.append(Rest_time_list[-1] - time - temp_df.loc[temp_df['code']==current_point]['상차시간(분)'].values.tolist()[0] - temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0])
				# 누적 시간 업데이트
				up_time= up_time+time+temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0] +temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0]
				# 누적 거리 업데이트
				
				# 기존 거리 = 기존 거리 + [DP-현재 위치]이동시간
				distance =distance+dist
				#print("거리",temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0],"  / 속도 :",temp_df.loc[temp_df['code']==current_point]['평균 운행 속도(km/h)'].values.tolist()[0])
				#print("소요시간 :",(temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]/temp_df.loc[temp_df['code']==current_point]['평균 운행 속도(km/h)'].values.tolist()[0])*60)
				next_check_time = (temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]/temp_df.loc[temp_df['code']==current_point]['평균 운행 속도(km/h)'].values.tolist()[0])*60
				# 현재 지점에서 DP까지 시간 여유
				print("누적거리 :   ",distance,"/ 누적시간  :        ",up_time,"/ time :        ",next_check_time , "possible stock :	",Rest_Stock_list[-1])
				print("Point_list : ", Point_list)
				print("From :",Point_list[-2],"To :",Point_list[-1])

			elif current_point !='DP':
				#print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>y!!!!!!DP")
				#거래처 위치 
				temp_df['x곡률값']=temp_df['경도(X좌표)']*3600
				temp_df['y곡률값']=temp_df['위도(Y좌표)']*3600

				#현재 위치 (DP >> 현재 위치 
				temp_df[current_point+'_x곡률값']=temp_df.loc[temp_df['code']==Point_list[-1]]['경도(X좌표)'].values.tolist()[0]*3600
				temp_df[current_point+'_y곡률값']=temp_df.loc[temp_df['code']==Point_list[-1]]['위도(Y좌표)'].values.tolist()[0]*3600

				#현재 위치와 거래처간의 계산
				temp_df[current_point+'위도거리']= abs(temp_df['x곡률값'] -temp_df[current_point+'_x곡률값'] )  *0.0245
				temp_df[current_point+'경도거리']= abs(temp_df['y곡률값'] -temp_df[current_point+'_y곡률값'] )  *0.0306
				temp_df[current_point+'운행거리'] = temp_df[current_point+'위도거리']+temp_df[current_point+'경도거리']
				#print(temp_df.head(1))			

				#print(temp_df.head(5))
				temp_df['각도']= np.degrees(np.arctan(temp_df[current_point+'위도거리']/temp_df[current_point+'경도거리']))

				#print("주행거리 : ",lowest_degree_row[current_point+'운행거리'])
				#distance =distance+ lowest_degree_row[current_point+'운행거리']


				#각도가 최적인 것을 선택
				#lowest_degree_row = temp_df[temp_df['stopby']==0].sort_values(by= '각도').iloc[0]
				tdf= temp_df[temp_df['stopby']==0]
				tdf = tdf.copy()
				lowest_degree_row = tdf.sort_values(by=current_point+'운행거리').iloc[0]

				dist = lowest_degree_row[current_point+'운행거리'].tolist()
				fast = lowest_degree_row['평균 운행 속도(km/h)'].tolist()
				time =  (dist / fast)*60
				print("dist :   ",dist,"/ fast :        ",fast,"/ time :        ",time)


				#현재f 위치 변경
				current_point = lowest_degree_row['code']
				Point_list.append(current_point)

				# stopby =1 check
				temp_df.loc[(temp_df['code']==current_point),'stopby']=1
				
				#current_point = lowest_degree_row['code']

				#남은시간 리스트 업데이트 
				#  마지막 남은시간- [ 기존거래처 - 갱신 거래처]이동거리 - 하차				
				Rest_time_list.append((Rest_time_list[-1] -time - temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0]))

				# 누적 시간 업데이트 
				get_ff_time = temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0]

				up_time= up_time + time + get_ff_time

				distance =distance+dist

				current_point_stock= lowest_degree_row[day].tolist()
				#남은 적재량 리스트 업데이트 
				Rest_Stock_list.append(Rest_Stock_list[-1]-current_point_stock)
				#print(" ++ currrent_p : ",current_point,"   current_stock : ",current_point_stock)
				print("누적거리 :   ",distance,"/ 누적시간  :        ",up_time, "/ Rest_time :	 ",Rest_time_list[-1],  " / possible stock :,",Rest_Stock_list[-1],"/ possible time :        ",next_check_time)
				print("From :",Point_list[-2],"To :",Point_list[-1])

				next_check_time = (temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]/temp_df.loc[temp_df['code']==current_point]['평균 운행 속도(km/h)'].values.tolist()[0])*60


			d_list.append(distance)

			if  Rest_time_list[-1]- next_check_time>0 and Rest_Stock_list[-1] >0  and up_time<360:
				print("조건 1",Rest_time_list[-1]- next_check_time,Rest_Stock_list[-1])
				#print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<",up_time)
				continue
			elif Rest_time_list[-1]- next_check_time>0 and Rest_Stock_list[-1] <=0 and up_time <360:
				print("조건 2",Rest_time_list[-1]- next_check_time,Rest_Stock_list[-1])
				# 여유시간이 충분한데 capa 가 없을 때 
				current_point ='DP'
				# 회차 하고 다시 다음 회전을 위한 상차 
				Rest_time_list.append((Rest_time_list[-1]- next_check_time - temp_df.iloc[0]['상차시간(분)']))
				
				up_time = up_time+next_check_time+temp_df.iloc[0]['상차시간(분)']
				Point_list.append(current_point)
				Rest_Stock_list.append(temp_df.iloc[0]['대당 적재능력(box)'])
				#distance = distance + temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]
				d_list.append(distance)
				print("					<<<<<<<<<<<<<회차 현황 : >>>>>>>>>>>>>>>>>>\n")
				print("누적거리 :   ",distance,"/ 누적시간  :        ",up_time, "/ Rest_time :   ",Rest_time_list[-1],  " / possible stock :,",Rest_Stock_list[-1],"/ possible time :        ",next_check_time)
				if up_time <360 or Rest_time_list[-1] > 0:
					print("*************************************************************************************************")
					continue
				
				elif up_time > 360 or Rest_time_list[-1] < 0 :
					current_point = 'DP'
					Point_list = ['DP']
					Rest_Stock_list = [temp_df.iloc[0]['대당 적재능력(box)']]
					Rest_time_list = [temp_df.iloc[0]['대당 운행시간(분)']]
					up_time =0
					
					print(" >>>>>>>>>>>>>>>>>>>>>회차 시 누적 시간 이 360 초과 여서 운행 종료 <<<<<<<<<<<<<<<<<<<<<<<<<<<\n")



				# 여유시간이 없을 때 
			elif Rest_time_list[-1]- next_check_time <0 or Rest_time_list[-1]>0: 
				print("조건 3",Rest_time_list[-1]- next_check_time,Rest_Stock_list[-1])
				if up_time < 360:
				# 여유시간이 없을 때 마지막 거래처는 갔다온다

					#거래처 위치
					temp_df['x곡률값']=temp_df['경도(X좌표)']*3600
					temp_df['y곡률값']=temp_df['위도(Y좌표)']*3600

					#현재 위치 (Cient >> 마지막 복귀
					temp_df[current_point+'_x곡률값']=temp_df.loc[temp_df['code']==Point_list[-1]]['경도(X좌표)'].values.tolist()[0]*3600
					temp_df[current_point+'_y곡률값']=temp_df.loc[temp_df['code']==Point_list[-1]]['위도(Y좌표)'].values.tolist()[0]*3600

					#현재 위치와 거래처간의 계산
					temp_df[current_point+'위도거리']= abs(temp_df['x곡률값'] -temp_df[current_point+'_x곡률값'] )  *0.0245
					temp_df[current_point+'경도거리']= abs(temp_df['y곡률값'] -temp_df[current_point+'_y곡률값'] )  *0.0306
					temp_df[current_point+'운행거리'] = temp_df[current_point+'위도거리']+temp_df[current_point+'경도거리']
					#print(temp_df.head(1))

					#print(temp_df.head(5))
					temp_df['각도']= np.degrees(np.arctan(temp_df[current_point+'위도거리']/temp_df[current_point+'경도거리']))

					#print("주행거리 : ",lowest_degree_row[current_point+'운행거리'])
					#distance =distance+ lowest_degree_row[current_point+'운행거리']


					#각도가 최적인 것을 선택
					#lowest_degree_row = temp_df[temp_df['stopby']==0].sort_values(by='각도').iloc[0]

					tdf= temp_df[temp_df['stopby']==0]
					tdf = tdf.copy()
					lowest_degree_row = tdf.sort_values(by=current_point+'운행거리').iloc[0]

					dist = lowest_degree_row[current_point+'운행거리'].tolist()
					fast = lowest_degree_row['평균 운행 속도(km/h)'].tolist()
					time =  (dist / fast)*60
					print("dist :   ",dist,"/ fast :        ",fast,"/ time :        ",time)


					#현재 위치 변경
					current_point = lowest_degree_row['code']
					Point_list.append(current_point)
					print("Point list : ", Point_list)
					# stopby =1 check
					temp_df.loc[(temp_df['code']==current_point),'stopby']=1

					#current_point = lowest_degree_row['code']

					#남은시간 리스트 업데이트
					#  마지막 남은시간- [ 기존거래처 - 갱신 거래처]이동거리 - 하차
					Rest_time_list.append(Rest_time_list[-1] -time - temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0])

					# 누적 시간 업데이트
					get_ff_time = temp_df.loc[temp_df['code']==current_point]['하차시간(분)'].values.tolist()[0]

					up_time= up_time + time + get_ff_time
					distance =distance+dist

					current_point_stock= lowest_degree_row[day].tolist()
					#남은 적재량 리스트 업데이트
					Rest_Stock_list.append(Rest_Stock_list[-1]-current_point_stock)
					#print(" ++ currrent_p : ",current_point,"   current_stock : ",current_point_stock)
					print("누적거리 :   ",distance,"/ 누적시간  :        ",up_time, "/ Rest_time :   ",Rest_time_list[-1], "/ possible stock :,",Rest_Stock_list[-1],"  / possible time :        ",next_check_time)
					print("From :",Point_list[-2],"To :",Point_list[-1])

					next_check_time = (temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]/temp_df.loc[temp_df['code']==current_point]['평균 운행 속도(km/h)'].values.tolist()[0])*60
					distance= distance+temp_df.loc[temp_df['code']==current_point]['DP_운행거리'].values.tolist()[0]
					# 회차  >>DP
					Rest_time_list.append(Rest_time_list[-1]-next_check_time)
					up_time= up_time+ next_check_time
					print("누적거리 :   ",distance,"/ 누적시간  :        ",up_time, "/ Rest_time :   ",Rest_time_list[-1], " / possible stock :,",Rest_Stock_list[-1],">> 복귀 완료 \n")
					print("\n")
					
				else:
					print(" >>>>>>>>>>>>>>>>>>>>>회차 시 누적 시간 이 360 초과 여서 운행 종료 <<<<<<<<<<<<<<<<<<<<<<<<<<<\n")
				

				# 회차만 
				Rest_time_list.append(Rest_time_list[-1]- next_check_time)
				print("VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV",distance)
				truck_list.append(Point_list)
				col_list.append(len(Point_list))
				one_truck_list.append(len(Point_list)-Point_list.count("DP"))
				time_list.append(distance)
				T_time_list.append(up_time)

				current_point = 'DP'
				print("From :",Point_list[-2],"To :", current_point)

				Point_list = ['DP']
				Rest_Stock_list = [temp_df.iloc[0]['대당 적재능력(box)']]
				Rest_time_list = [temp_df.iloc[0]['대당 운행시간(분)']]
				DP_count.append(Point_list.count("DP")-1)
				#one_truck_list.append(len(Point_list))
				distnace = 0
				up_time =0

			if up_time> 360:
				distance =0
				print("VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV",distance)
				#one_truck_list.append(len(Point_list))
				#truck_list.append(Point_list)
				current_point = 'DP'
				Rest_Stock_list = [temp_df.iloc[0]['대당 적재능력(box)']]
				Rest_time_list = [temp_df.iloc[0]['대당 운행시간(분)']]
				up_time =0



		
		Max_row = max(col_list)
		temp_df.to_csv("temp1.csv", index=False, encoding='utf-8-sig')		
		result1 = pd.DataFrame(truck_list,columns = ["Client-"+str(i) for i in range(Max_row)])
		result2 = pd.DataFrame(one_truck_list,columns=["Client Count"])
		result3 = pd.DataFrame(time_list,columns=["누적 KM"])
		per_dist =[]
		take = result3["누적 KM"].values.tolist()
		for i in range(0, len(take)):
			
			if i == 0:
				per_dist.append(take[i])
			else:
				per_dist.append((take[i]-take[i-1]))

		result3 = pd.DataFrame(per_dist,columns=["KM"])
		result5 = pd.DataFrame(T_time_list,columns=["Time"])
		result6 = pd.DataFrame(DP_count,columns=["DP를 몇 번"])
		
		result4 = pd.concat([result1,result2,result3,result5, result6],axis=1)
		result4["Region"]= DF_element
		result4.to_csv("./"+day+"/"+DF_element+day+"complete.csv",encoding='utf-8-sig')
		#print(result)
		total_list =[]
		counting_list = []
		No_truck =1
		path_sum =[]

dist :    1.4809453954895753 / fast :         50 / time :         1.7771344745874904
누적거리 :    4751.5608570396025 / 누적시간  :         138.55252576706448 / Rest_time :	  216.44747423293555  / possible stock :, 97.0 / possible time :         75.21471508335709
From : 대구-220 To : 대구-218
조건 1 143.00989362416595 97.0
Point_list :  ['DP', '대구-198', '대구-220', '대구-218']
---------------------------------------------------------------------------------------------------------------------------------

dist :    0.45215387247356414 / fast :         50 / time :         0.542584646968277
누적거리 :    4752.013010912076 / 누적시간  :         154.09511041403275 / Rest_time :	  200.90488958596728  / possible stock :, 96.0 / possible time :         73.43758060876961
From : 대구-218 To : 대구-197
조건 1 127.13819011333887 96.0
Point_list :  ['DP', '대구-198', '대구-220', '대구-218', '대구-197']
---------------------------------------------------------------------------------------------------------------------------------

dist 

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
